In [1]:
!pip install datasets

In [2]:
!pip install transformers

In [3]:
!pip install scikit-multilearn

In [4]:
# !nvidia-smi

In [5]:
import pandas as pd

from transformers import BertForSequenceClassification, BertTokenizer, BertConfig, BertModel
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig

from skmultilearn.model_selection import iterative_train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.cluster import contingency_matrix

import shutil
import os
import torch
import json
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
np.random.seed(42)


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model helper functions and classes

In [6]:
def pre_process_data(df, test_proportion, train_size):

    df.columns.values[0] = "FQText"
    size_df = df.shape[0]
    df = shuffle(df, random_state = 42)

    X = df[["FQText"]].to_numpy()

    y_df = df.drop(["FQText"], axis=1).astype(np.float32)
    cols = df.columns
    df_labels = list(y_df.columns)

    y = y_df.to_numpy()
    print(y_df.info())

    test_proportion = 0.05
    train_size = 0.9

    X_model_train, y_model_train, X_test, y_test = iterative_train_test_split(X, y, test_size=test_proportion)
    X_train, y_train, X_val, y_val = iterative_train_test_split(X_model_train, y_model_train, test_size= 1-train_size)

    df_train = pd.concat([pd.DataFrame(X_train, columns=["FQText"]), pd.DataFrame(y_train, columns=[df_labels], dtype=np.float32)], axis=1).reset_index(drop=True)
    df_val = pd.concat([pd.DataFrame(X_val, columns=["FQText"]), pd.DataFrame(y_val, columns=[df_labels], dtype=np.float32)], axis=1).reset_index(drop=True)
    df_test = pd.concat([pd.DataFrame(X_test, columns=["FQText"]), pd.DataFrame(y_test, columns=[df_labels], dtype=np.float32)], axis=1).reset_index(drop=True)

    df_train.columns = cols
    df_val.columns = cols
    df_test.columns = cols

    print(df_labels)

    # train_df = df_train.sample(frac=train_size, random_state=42).reset_index(drop=True)
    # val_df = df_train.drop(train_df.index).reset_index(drop=True)

    print("Total amount of data: {}".format(size_df))
    print("Number of rows used to TRAIN: {}".format(df_train.shape[0]))
    print("Number of rows used to VALIDATE: {}".format(df_val.shape[0]))
    print("Number of rows used to TEST: {}".format(df_test.shape[0]))

    return df_train, df_val, df_test, df_labels

In [7]:
def set_hyperparams(hp_dictionary):

    MAX_LEN = hp_dictionary["MAX_LEN"]
    TRAIN_BATCH_SIZE = hp_dictionary["TRAIN_BATCH_SIZE"]
    VALID_BATCH_SIZE = hp_dictionary["VALID_BATCH_SIZE"]
    EPOCHS = hp_dictionary["EPOCHS"]
    LEARNING_RATE = hp_dictionary["LEARNING_RATE"]

    return MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE

In [8]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [9]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['FQText']
        # self.labels = list(df.columns)[2:] # list of the target values
        self.targets = self.df[df_labels].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True, # do we actually need special tokens ??
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'  # pytorch tensors
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [10]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        # self.bert_model = LongformerModel.from_pretrained('allenai/longformer-base-4096', return_dict=True, problem_type="multi_label_classification")
        self.bert_model = BertModel.from_pretrained("bert-base-uncased", return_dict=True, problem_type="multi_label_classification")

        self.dropout = torch.nn.Dropout(0.3) # why 0.3?
        self.linear = torch.nn.Linear(768, len(df_labels)) # have to changet he n of possible labels here

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )

        # the issue is that bert gets size 6 here?
        output_dropout = self.dropout(output.pooler_output)
        # print(output_dropout)
        output = self.linear(output_dropout)
        return output

In [11]:
def loss_fn(outputs, targets):
    # print(outputs, targets)
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

## Train model function

In [12]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path, df_labels):
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
  best_metrics = {}
  # best_metrics[f"n_epochs"] = n_epochs + 1 # HERE IS THE ISSUE
  
  best_metrics[f"n_epochs"] = n_epochs 


  # val_targets_study= []
  # val_outputs_study= [] # I am going to extend all the validation probabilities to determine what the prob threshold should be

  for epoch in range(1, n_epochs+1):

    epoch_n = "epoch_{}".format(epoch - 1) #ISSUE MAY HAVE BEEN HERE TOO, IM CHANGIN TO TO START AT EPOCH 0 AND JUST DO INDEXING STARTING AT 0
    best_metrics[epoch_n] = {}

    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        # print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        """targets are the y array of the original data.
        """

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)



    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    val_targets = []
    val_outputs = []

    proba_outputs = []


    sigmoid = torch.nn.Sigmoid()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
        val_targets.extend(targets.cpu().detach().numpy().tolist())
        val_outputs.extend(outputs.cpu().detach().numpy().tolist())

        probs = sigmoid(torch.Tensor(outputs))
        proba_outputs.append(probs.cpu().detach().numpy().tolist())

        # # print(probs)

        # val_targets_study.append(targets.cpu().detach().numpy().tolist())
        # val_outputs_study.append(probs.cpu().detach().numpy().tolist())



      print('############# Epoch {}  Metrics   #############\n\n'.format(epoch))
      metrics, f1, roc_auc, hamming, accuracy = multi_labels_metrics(val_outputs, val_targets, df_labels, threshold=0.5)

      print('############# Epoch {}: Validation End     #############'.format(epoch))

      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)


      best_metrics[epoch_n][f"train_loss_epoch_{epoch}"] = train_loss
      best_metrics[epoch_n][f"valid_loss_epoch_{epoch}"] = valid_loss
      best_metrics[epoch_n][f"f1_epoch_{epoch}"] = f1
      best_metrics[epoch_n][f"accuracy_epoch_{epoch}"] = accuracy
      best_metrics[epoch_n][f"roc_auc_epoch_{epoch}"] = roc_auc
      best_metrics[epoch_n][f"hamming_epoch_{epoch}"] = hamming
      best_metrics[epoch_n][f"proba_outputs_epoch_{epoch}"] = proba_outputs
      best_metrics[epoch_n][f"actual_classes_epoch_{epoch}"] = val_targets

      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data

      # checkpoint = {
      #       'epoch': epoch + 1,
      #       'valid_loss_min': valid_loss,
      #       'state_dict': model.state_dict(),
      #       'optimizer': optimizer.state_dict()
      # }

        # save checkpoint
      # save_ckp(checkpoint, True, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        # save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss
        best_metrics["best_metrics"] = {
                'epoch': epoch,
                'train_loss': train_loss,
                'valid_loss': valid_loss,
                'val_acc': accuracy,
                'val_f1': f1,
                'val_roc_auc': roc_auc,
                'val_hamming': hamming,
                'proba_outputs': proba_outputs,
                'actual_classes': val_targets
            }

    # print((f"EVAL METRICS: {metrics}\n"))

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model, best_metrics

In [13]:
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss, accuracy_score

def multi_labels_metrics(predictions, targets, df_labels, threshold=0.36): # mess with threshold

  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions)) #mapply activation function in the raw values

  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs>=threshold)] = 1
  y_true = targets

  accuracy = accuracy_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred, average = 'micro')
  roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
  hamming = hamming_loss(y_true, y_pred)
  class_report = classification_report(y_true, y_pred, target_names=df_labels)

  metrics = {
      "roc_auc": roc_auc,       # special multilabel metrics
      "hamming_loss": hamming,  # special multilabel metrics
      "f1": f1,
      "accuracy_score": accuracy
  }
  print(metrics)
  print(class_report)
  return metrics, f1, roc_auc, hamming, accuracy

def compute_test_metrics(test_loader):

  model.eval()

  test_loss = 0
  metrics_targets = []
  metrics_outputs = []
  metrics_proba = []

  sigmoid = torch.nn.Sigmoid()
  with torch.no_grad():
        for _, data in enumerate(test_loader, 0):
              ids = data['input_ids'].to(device, dtype = torch.long)
              mask = data['attention_mask'].to(device, dtype = torch.long)
              token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
              targets = data['targets'].to(device, dtype = torch.float)

              outputs = model(ids.squeeze(), mask.squeeze(), token_type_ids.squeeze())

              loss = loss_fn(outputs, targets)
              test_loss = test_loss + ((1 / (_ + 1)) * (loss.item() - test_loss))

              metrics_targets.extend(targets.cpu().detach().numpy().tolist())
              metrics_outputs.extend(outputs.cpu().detach().numpy().tolist())


              probs = sigmoid(torch.Tensor(outputs))

              # val_targets_study.append(targets.cpu().detach().numpy().tolist())
              metrics_proba.append(probs.cpu().detach().numpy().tolist())

  test_loss = test_loss/len(test_loader)

  metrics, f1, roc_auc, hamming, accuracy = multi_labels_metrics(metrics_outputs, metrics_targets, df_labels)
  metrics["test_loss"] = test_loss


  return metrics, f1, roc_auc, hamming, accuracy, test_loss, metrics_proba, metrics_targets


# GRIDSEARCH

In [14]:
# original param_grid

'''hp_dictionary = {"MAX_LEN":256 ,
    "TRAIN_BATCH_SIZE": 64,
    "VALID_BATCH_SIZE": 64,
    "EPOCHS": 4,
    "LEARNING_RATE": 1e-05
}'''

'hp_dictionary = {"MAX_LEN":256 ,\n    "TRAIN_BATCH_SIZE": 64,\n    "VALID_BATCH_SIZE": 64,\n    "EPOCHS": 4,\n    "LEARNING_RATE": 1e-05\n}'

In [15]:
'''for batch in batch_size:
  for rate in learning_rate:
    for epoch in num_epochs:
      hp_dictionary = {"MAX_LEN":256 ,
        "TRAIN_BATCH_SIZE": batch,
        "VALID_BATCH_SIZE": batch,
        "EPOCHS": epoch,
        "LEARNING_RATE": rate}
      hp_dictionaries.append(hp_dictionary)'''

'for batch in batch_size:\n  for rate in learning_rate:\n    for epoch in num_epochs:\n      hp_dictionary = {"MAX_LEN":256 ,\n        "TRAIN_BATCH_SIZE": batch,\n        "VALID_BATCH_SIZE": batch,\n        "EPOCHS": epoch,\n        "LEARNING_RATE": rate}\n      hp_dictionaries.append(hp_dictionary)'

In [16]:
import itertools

In [17]:
#max-len : [128]
#batch_size = [8,12,16,32,64]
#learning_rate = [2e-5, 3e-5, 5e-5]
#num_epochs = [2, 3, 4]

In [18]:
#initialize empty list to hold all hyperparameter combinations
hp_dictionaries = []

#build the grid
#LOW GRID NUMBERS FOR TESTING
#USE SETS ABOVE

# param_grid = {
#     'max-len' : [128, 256, 512],
#     'learning_rate': [1e-5, 2e-5, 3e-5, 5e-5],
#     'batch_size': [8, 12, 16, 32, 64],
#     'num_train_epochs': [16]
# }

# UPDATED PARAM GRID AFTER FIRST SEARCH
param_grid = {
    'max-len' : [128],
    'learning_rate': [2e-5, 3e-5, 5e-5],
    'batch_size': [8, 12, 16],
    'num_train_epochs': [14]
}

#create cartesian product of the parameters
param_combos = list(itertools.product(*param_grid.values()))

#add the combinations into the list of dictionaries
for params in param_combos:
    hp_dictionaries.append(dict(zip(param_grid.keys(), params)))


# Define the function to set hyperparameters
def set_hyperparams(param_grid):
    MAX_LEN = param_grid['max-len']
    TRAIN_BATCH_SIZE = param_grid['batch_size']
    VALID_BATCH_SIZE = param_grid['batch_size']
    EPOCHS = param_grid['num_train_epochs']
    LEARNING_RATE = param_grid['learning_rate']
    return MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE

#print statement for testing to see the cartesian product functions
print(hp_dictionaries)
print(len(hp_dictionaries))

[{'max-len': 128, 'learning_rate': 2e-05, 'batch_size': 8, 'num_train_epochs': 14}, {'max-len': 128, 'learning_rate': 2e-05, 'batch_size': 12, 'num_train_epochs': 14}, {'max-len': 128, 'learning_rate': 2e-05, 'batch_size': 16, 'num_train_epochs': 14}, {'max-len': 128, 'learning_rate': 3e-05, 'batch_size': 8, 'num_train_epochs': 14}, {'max-len': 128, 'learning_rate': 3e-05, 'batch_size': 12, 'num_train_epochs': 14}, {'max-len': 128, 'learning_rate': 3e-05, 'batch_size': 16, 'num_train_epochs': 14}, {'max-len': 128, 'learning_rate': 5e-05, 'batch_size': 8, 'num_train_epochs': 14}, {'max-len': 128, 'learning_rate': 5e-05, 'batch_size': 12, 'num_train_epochs': 14}, {'max-len': 128, 'learning_rate': 5e-05, 'batch_size': 16, 'num_train_epochs': 14}]
9


In [19]:
grid_metric_results = []

In [20]:
failed_grids = []

TRAIN AND VALIDATION LOOP

In [21]:
# dfs = ['nancy_determinants_individual_labels_eng.csv']

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [24]:
def train_final_models(param_grid_list, n_grids, train_df, val_df, df_test, df_labels):
  i = 0
  grid_metric_results = []
  for hp_dictionary in param_grid_list:
    print(f"PARAM {i}/{n_grids}")
    print(hp_dictionary)

    # try:

    MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE = set_hyperparams(hp_dictionary)

    train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
    valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
    test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN)

    train_data_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True,
        num_workers=0
    )

    val_data_loader = torch.utils.data.DataLoader(valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    test_data_loader = torch.utils.data.DataLoader(test_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    model = BERTClass()
    model.to(device)
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


    ckpt_path = "curr_ckpt"
    best_model_path = "best_model.pt"

    trained_model, best_metrics = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path, df_labels)

    metrics, f1, roc_auc, hamming, accuracy, test_loss, metrics_proba, metrics_targets = compute_test_metrics(test_data_loader)

    best_metrics["test"] = {}
    best_metrics["test"]["test_loss"] = test_loss
    best_metrics["test"]["test_f1"] = f1
    best_metrics["test"]["test_roc_auc"] = roc_auc
    best_metrics["test"]["test_hamming"] = hamming
    best_metrics["test"]["test_accuracy"] = accuracy
    best_metrics["test"]["test_proba"] = metrics_proba
    best_metrics["test"]["test_actual_classes"] = metrics_targets

    result = {
          'hyperparameters': hp_dictionary,
          'metrics': best_metrics
      }

    grid_metric_results.append(result)
    i = i + 1

  return grid_metric_results

In [25]:
dfs = ['nancy_determinants_individual_labels_eng.csv',
 'nancy_contents_individual_labels_eng.csv',
 'nancy_contents_macro_labels_english.csv',
 'nancy_determinants_macro_labels_english.csv']

# Individual Determinants

In [26]:
param_grid_ind_dets = [
                        {
                        'max-len' : 128,
                        'learning_rate': 3e-5,
                        'batch_size': 8,
                        'num_train_epochs': 40
                                                },
                        {
                        'max-len' : 128,
                        'learning_rate': 5e-5,
                        'batch_size': 8,
                        'num_train_epochs': 40
                                                }]

n_grids = len(param_grid_ind_dets)

In [102]:
def n_tokens(row):
    tokens = tokenizer.encode(row)
    len_tokens = len(tokens)
    return len_tokens

In [80]:
df_name = dfs[0]
data_model = df_name.split("_")
data_model = "_".join(data_model[1:3]) #getting the name to name the different dictionaries
df = pd.read_csv(df_name)

test_proportion = 0.05
train_size = 0.9

train_df, val_df, df_test, df_labels  = pre_process_data(df, test_proportion, train_size)

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, 266 to 102
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   C       380 non-null    float32
 1   C'      380 non-null    float32
 2   C'F     380 non-null    float32
 3   CF      380 non-null    float32
 4   CF'     380 non-null    float32
 5   CLOB    380 non-null    float32
 6   CLOBF   380 non-null    float32
 7   E       380 non-null    float32
 8   EF      380 non-null    float32
 9   F       380 non-null    float32
 10  FC      380 non-null    float32
 11  FC'     380 non-null    float32
 12  FCLOB   380 non-null    float32
 13  FE      380 non-null    float32
 14  K       380 non-null    float32
 15  KAN     380 non-null    float32
 16  KOB     380 non-null    float32
 17  KP      380 non-null    float32
dtypes: float32(18)
memory usage: 29.7 KB
None
['C', "C'", "C'F", 'CF', "CF'", 'CLOB', 'CLOBF', 'E', 'EF', 'F', 'FC', "FC'", 'FCLOB', 'FE', 'K', 'KAN', 'KOB',

In [108]:
# first_col = df.columns[0]
# df["num_tokens"] = df[first_col].apply(n_tokens)

## Distribution of the classes in train, validate, test dfs


In [81]:
def count_class(df):
    distribution_class = {}
    for col in df.columns[1:]:
        # print(type(train_df[col].value_counts()))
        # print(amount_class.index)
        # print(amount_class[1.0])
        try:
            amount_class = df[col].value_counts()[1.0]
        except:
            amount_class = "no positive class"
        
        distribution_class[col] = amount_class
    return distribution_class

Train, validate and test splits


In [82]:
distribution_class_training = count_class(train_df)
distribution_class_training

{'C': 32,
 "C'": 21,
 "C'F": 2,
 'CF': 14,
 "CF'": 2,
 'CLOB': 14,
 'CLOBF': 4,
 'E': 24,
 'EF': 5,
 'F': 129,
 'FC': 28,
 "FC'": 20,
 'FCLOB': 5,
 'FE': 7,
 'K': 38,
 'KAN': 29,
 'KOB': 20,
 'KP': 12}

In [83]:
distribution_class_val = count_class(val_df)
distribution_class_val

{'C': 3,
 "C'": 2,
 "C'F": 1,
 'CF': 1,
 "CF'": 'no positive class',
 'CLOB': 1,
 'CLOBF': 'no positive class',
 'E': 3,
 'EF': 1,
 'F': 14,
 'FC': 3,
 "FC'": 2,
 'FCLOB': 'no positive class',
 'FE': 1,
 'K': 4,
 'KAN': 3,
 'KOB': 2,
 'KP': 1}

In [84]:
distribution_class_training = count_class(df_test)
distribution_class_training

{'C': 2,
 "C'": 1,
 "C'F": 'no positive class',
 'CF': 1,
 "CF'": 'no positive class',
 'CLOB': 1,
 'CLOBF': 'no positive class',
 'E': 1,
 'EF': 'no positive class',
 'F': 7,
 'FC': 2,
 "FC'": 1,
 'FCLOB': 'no positive class',
 'FE': 'no positive class',
 'K': 2,
 'KAN': 2,
 'KOB': 1,
 'KP': 1}

## Training the models

In [24]:
i = 0
grid_metric_results = []
for hp_dictionary in param_grid_ind_dets:
    print(f"PARAM {i}/{n_grids}")
    print(hp_dictionary)

    # try:

    MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE = set_hyperparams(hp_dictionary)

    train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
    valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
    test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN)

    train_data_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True,
        num_workers=0
    )

    val_data_loader = torch.utils.data.DataLoader(valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    test_data_loader = torch.utils.data.DataLoader(test_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    model = BERTClass()
    model.to(device)
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


    ckpt_path = "curr_ckpt"
    best_model_path = "best_model.pt"

    trained_model, best_metrics = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path, df_labels)

    metrics, f1, roc_auc, hamming, accuracy, test_loss, metrics_proba, metrics_targets = compute_test_metrics(test_data_loader)

    best_metrics["test"] = {}
    best_metrics["test"]["test_loss"] = test_loss
    best_metrics["test"]["test_f1"] = f1
    best_metrics["test"]["test_roc_auc"] = roc_auc
    best_metrics["test"]["test_hamming"] = hamming
    best_metrics["test"]["test_accuracy"] = accuracy
    best_metrics["test"]["test_proba"] = metrics_proba
    best_metrics["test"]["test_actual_classes"] = metrics_targets

    result = {
          'hyperparameters': hp_dictionary,
          'metrics': best_metrics
      }

    grid_metric_results.append(result)
    i = i + 1

PARAM 0/2
{'max-len': 128, 'learning_rate': 3e-05, 'batch_size': 8, 'num_train_epochs': 40}


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.06481481481481481, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
          C'       0.00      0.00      0.00         2
         C'F       0.00      0.00      0.00         1
          CF       0.00      0.00      0.00         1
         CF'       0.00      0.00      0.00         0
        CLOB       0.00      0.00      0.00         1
       CLOBF       0.00      0.00      0.00         0
           E       0.00      0.00      0.00         3
          EF       0.00      0.00      0.00         1
           F       0.00      0.00      0.00        14
          FC       0.00      0.00      0.00         3
         FC'       0.00      0.00      0.00       

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.06481481481481481, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
          C'       0.00      0.00      0.00         2
         C'F       0.00      0.00      0.00         1
          CF       0.00      0.00      0.00         1
         CF'       0.00      0.00      0.00         0
        CLOB       0.00      0.00      0.00         1
       CLOBF       0.00      0.00      0.00         0
           E       0.00      0.00      0.00         3
          EF       0.00      0.00      0.00         1
           F       0.00      0.00      0.00        14
          FC       0.00      0.00      0.00         3
         FC'       0.00      0.00      0.00         2
       FCLOB       0.00      0.00      0.00       

...

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

############# Epoch 40: Training End     #############
############# Epoch 40: Validation Start   #############
############# Epoch 40  Metrics   #############


{'roc_auc': 0.7417491749174917, 'hamming_loss': 0.047839506172839504, 'f1': 0.5753424657534247, 'accuracy_score': 0.4722222222222222}
              precision    recall  f1-score   support

           C       0.00      0.00      0.00         3
          C'       1.00      0.50      0.67         2
         C'F       0.00      0.00      0.00         1
          CF       1.00      1.00      1.00         1
         CF'       0.00      0.00      0.00         0
        CLOB       0.00      0.00      0.00         1
       CLOBF       0.00      0.00      0.00         0
           E       1.00      0.33      0.50         3
          EF       0.00      0.00      0.00         1
           F       0.71      0.86      0.77        14
          FC       0.00      0.00      0.00         3
         FC'       0.00      0.00      0.00         2
 

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

{'roc_auc': 0.6852664172364855, 'hamming_loss': 0.058333333333333334, 'f1': 0.46153846153846156, 'accuracy_score': 0.35}
              precision    recall  f1-score   support

           C       1.00      0.50      0.67         2
          C'       0.00      0.00      0.00         1
         C'F       0.00      0.00      0.00         0
          CF       0.00      0.00      0.00         1
         CF'       0.00      0.00      0.00         0
        CLOB       0.00      0.00      0.00         1
       CLOBF       0.00      0.00      0.00         0
           E       0.00      0.00      0.00         1
          EF       0.00      0.00      0.00         0
           F       0.60      0.38      0.46         8
          FC       0.00      0.00      0.00         2
         FC'       0.25      1.00      0.40         1
       FCLOB       0.00      0.00      0.00         0
          FE       0.00      0.00      0.00         0
           K       1.00      1.00      1.00         2
         KAN  

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

In [25]:
with open("metrics_ind_dets_final.json", "w") as f:
    json.dump(grid_metric_results, f)

# Individual Contents

In [85]:
param_grid_ind_conts = [
                        {
                        'max-len' : 128,
                        'learning_rate': 3e-5,
                        'batch_size': 8,
                        'num_train_epochs': 40
                                                },
                        {
                        'max-len' : 128,
                        'learning_rate': 5e-5,
                        'batch_size': 12,
                        'num_train_epochs': 40
                                                }]
n_grids = len(param_grid_ind_conts)

In [86]:
df_name = dfs[1]
data_model = df_name.split("_")
data_model = "_".join(data_model[1:3]) #getting the name to name the different dictionaries
df = pd.read_csv(df_name)

test_proportion = 0.05
train_size = 0.9

train_df, val_df, df_test, df_labels  = pre_process_data(df, test_proportion, train_size)

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, 266 to 102
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   (A)     380 non-null    float32
 1   (AD)    380 non-null    float32
 2   (H)     380 non-null    float32
 3   (HD)    380 non-null    float32
 4   A       380 non-null    float32
 5   ABS     380 non-null    float32
 6   AD      380 non-null    float32
 7   ALIM    380 non-null    float32
 8   ANAT    380 non-null    float32
 9   ARCH    380 non-null    float32
 10  ART     380 non-null    float32
 11  BOT     380 non-null    float32
 12  ELEM    380 non-null    float32
 13  FRAG    380 non-null    float32
 14  GÉO     380 non-null    float32
 15  H       380 non-null    float32
 16  HD      380 non-null    float32
 17  MQ      380 non-null    float32
 18  NAT     380 non-null    float32
 19  OBJ     380 non-null    float32
 20  PAYS    380 non-null    float32
 21  RADIO   380 non-null    float32
 22  SC   

## Distribution of the classes in train, validate, test dfs

In [87]:
distribution_class_training = count_class(train_df)
distribution_class_training

{'(A)': 10,
 '(AD)': 5,
 '(H)': 15,
 '(HD)': 6,
 'A': 95,
 'ABS': 15,
 'AD': 40,
 'ALIM': 5,
 'ANAT': 18,
 'ARCH': 7,
 'ART': 4,
 'BOT': 16,
 'ELEM': 20,
 'FRAG': 20,
 'GÉO': 2,
 'H': 31,
 'HD': 40,
 'MQ': 4,
 'NAT': 7,
 'OBJ': 36,
 'PAYS': 15,
 'RADIO': 1,
 'SC': 1,
 'SCÈNE': 3,
 'SEX': 6,
 'SG': 10,
 'VÊT': 11}

In [88]:
distribution_class_validation = count_class(val_df)
distribution_class_validation

{'(A)': 1,
 '(AD)': 'no positive class',
 '(H)': 2,
 '(HD)': 1,
 'A': 10,
 'ABS': 2,
 'AD': 4,
 'ALIM': 1,
 'ANAT': 2,
 'ARCH': 1,
 'ART': 'no positive class',
 'BOT': 2,
 'ELEM': 2,
 'FRAG': 2,
 'GÉO': 'no positive class',
 'H': 3,
 'HD': 4,
 'MQ': 'no positive class',
 'NAT': 1,
 'OBJ': 4,
 'PAYS': 2,
 'RADIO': 'no positive class',
 'SC': 'no positive class',
 'SCÈNE': 'no positive class',
 'SEX': 1,
 'SG': 1,
 'VÊT': 1}

In [89]:
distribution_class_test = count_class(df_test)
distribution_class_test

{'(A)': 1,
 '(AD)': 'no positive class',
 '(H)': 1,
 '(HD)': 'no positive class',
 'A': 6,
 'ABS': 1,
 'AD': 2,
 'ALIM': 'no positive class',
 'ANAT': 1,
 'ARCH': 'no positive class',
 'ART': 'no positive class',
 'BOT': 1,
 'ELEM': 1,
 'FRAG': 1,
 'GÉO': 'no positive class',
 'H': 2,
 'HD': 2,
 'MQ': 'no positive class',
 'NAT': 'no positive class',
 'OBJ': 2,
 'PAYS': 1,
 'RADIO': 'no positive class',
 'SC': 'no positive class',
 'SCÈNE': 'no positive class',
 'SEX': 'no positive class',
 'SG': 1,
 'VÊT': 1}

# Training the models

In [28]:
i = 0
grid_metric_results = []
for hp_dictionary in param_grid_ind_conts:
    print(f"PARAM {i}/{n_grids}")
    print(hp_dictionary)

    # try:

    MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE = set_hyperparams(hp_dictionary)

    train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
    valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
    test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN)

    train_data_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True,
        num_workers=0
    )

    val_data_loader = torch.utils.data.DataLoader(valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    test_data_loader = torch.utils.data.DataLoader(test_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    model = BERTClass()
    model.to(device)
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


    ckpt_path = "curr_ckpt"
    best_model_path = "best_model.pt"

    trained_model, best_metrics = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path, df_labels)

    metrics, f1, roc_auc, hamming, accuracy, test_loss, metrics_proba, metrics_targets = compute_test_metrics(test_data_loader)

    best_metrics["test"] = {}
    best_metrics["test"]["test_loss"] = test_loss
    best_metrics["test"]["test_f1"] = f1
    best_metrics["test"]["test_roc_auc"] = roc_auc
    best_metrics["test"]["test_hamming"] = hamming
    best_metrics["test"]["test_accuracy"] = accuracy
    best_metrics["test"]["test_proba"] = metrics_proba
    best_metrics["test"]["test_actual_classes"] = metrics_targets

    result = {
          'hyperparameters': hp_dictionary,
          'metrics': best_metrics
      }

    grid_metric_results.append(result)
    i = i + 1

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


PARAM 0/2
{'max-len': 128, 'learning_rate': 3e-05, 'batch_size': 8, 'num_train_epochs': 40}
############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.04245709123757904, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

         (A)       0.00      0.00      0.00         1
        (AD)       0.00      0.00      0.00         0
         (H)       0.00      0.00      0.00         2
        (HD)       0.00      0.00      0.00         1
           A       0.00      0.00      0.00        10
         ABS       0.00      0.00      0.00         2
          AD       0.00      0.00      0.00         4
        ALIM       0.00      0.00      0.00         1
        ANAT       0.00      0.00      0.00         2
        ARCH       0.00      0.00      0.00         1
         ART

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.04245709123757904, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

         (A)       0.00      0.00      0.00         1
        (AD)       0.00      0.00      0.00         0
         (H)       0.00      0.00      0.00         2
        (HD)       0.00      0.00      0.00         1
           A       0.00      0.00      0.00        10
         ABS       0.00      0.00      0.00         2
          AD       0.00      0.00      0.00         4
        ALIM       0.00      0.00      0.00         1
        ANAT       0.00      0.00      0.00         2
        ARCH       0.00      0.00      0.00         1
         ART       0.00      0.00      0.00         0
         BOT       0.00      0.00      0.00         2
        ELEM       0.00      0.00      0.00       

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

...

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

############# Epoch 40: Training End     #############
############# Epoch 40: Validation Start   #############
############# Epoch 40  Metrics   #############


{'roc_auc': 0.7699919710959455, 'hamming_loss': 0.031616982836495035, 'f1': 0.5977011494252874, 'accuracy_score': 0.4634146341463415}
              precision    recall  f1-score   support

         (A)       1.00      1.00      1.00         1
        (AD)       0.00      0.00      0.00         0
         (H)       0.00      0.00      0.00         2
        (HD)       0.00      0.00      0.00         1
           A       0.80      0.80      0.80        10
         ABS       1.00      0.50      0.67         2
          AD       0.44      1.00      0.62         4
        ALIM       1.00      1.00      1.00         1
        ANAT       0.00      0.00      0.00         2
        ARCH       0.00      0.00      0.00         1
         ART       0.00      0.00      0.00         0
         BOT       1.00      0.50      0.67         2
 

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

{'roc_auc': 0.8266331658291457, 'hamming_loss': 0.02576489533011272, 'f1': 0.6666666666666666, 'accuracy_score': 0.5652173913043478}
              precision    recall  f1-score   support

         (A)       1.00      1.00      1.00         1
        (AD)       0.00      0.00      0.00         0
         (H)       1.00      1.00      1.00         1
        (HD)       0.00      0.00      0.00         0
           A       1.00      1.00      1.00         6
         ABS       0.00      0.00      0.00         1
          AD       0.33      0.50      0.40         2
        ALIM       0.00      0.00      0.00         0
        ANAT       0.00      0.00      0.00         1
        ARCH       0.00      0.00      0.00         0
         ART       0.00      0.00      0.00         0
         BOT       0.33      1.00      0.50         1
        ELEM       0.33      1.00      0.50         1
        FRAG       0.00      0.00      0.00         1
         GÉO       0.00      0.00      0.00         0
  

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

In [29]:
with open("metrics_ind_conts_final.json", "w") as f:
    json.dump(grid_metric_results, f)

# Macro Contents

In [30]:
param_grid_macro_cont = [
                        {
                        'max-len' : 128,
                        'learning_rate': 3e-5,
                        'batch_size': 8,
                        'num_train_epochs': 40
                                                },
                        {
                        'max-len' : 128,
                        'learning_rate': 5e-5,
                        'batch_size': 8,
                        'num_train_epochs': 40
                                                },
                         {
                        'max-len' : 128,
                        'learning_rate': 2e-5,
                        'batch_size': 8,
                        'num_train_epochs': 40
                                                },
                        {
                        'max-len' : 128,
                        'learning_rate': 5e-5,
                        'batch_size': 12,
                        'num_train_epochs': 40
                                                }]
n_grids = len(param_grid_macro_cont)

In [90]:
df_name = dfs[2]
data_model = df_name.split("_")
data_model = "_".join(data_model[1:3]) #getting the name to name the different dictionaries
df = pd.read_csv(df_name)

test_proportion = 0.05
train_size = 0.9

train_df, val_df, df_test, df_labels  = pre_process_data(df, test_proportion, train_size)

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, 266 to 102
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   animal_sum    380 non-null    float32
 1   human_sum     380 non-null    float32
 2   abs_sum       380 non-null    float32
 3   food_sum      380 non-null    float32
 4   art_arch_sum  380 non-null    float32
 5   nature_sum    380 non-null    float32
 6   fragment_sum  380 non-null    float32
 7   geo_sum       380 non-null    float32
 8   object_sum    380 non-null    float32
 9   science_sum   380 non-null    float32
 10  graphic_sum   380 non-null    float32
dtypes: float32(11)
memory usage: 19.3 KB
None
['animal_sum', 'human_sum', 'abs_sum', 'food_sum', 'art_arch_sum', 'nature_sum', 'fragment_sum', 'geo_sum', 'object_sum', 'science_sum', 'graphic_sum']
Total amount of data: 380
Number of rows used to TRAIN: 321
Number of rows used to VALIDATE: 37
Number of rows used to TEST: 22


## Distribution of the classes in train, validate, test dfs


In [91]:
distribution_class_training = count_class(train_df)
distribution_class_training


{'animal_sum': 153,
 'human_sum': 85,
 'abs_sum': 15,
 'food_sum': 5,
 'art_arch_sum': 12,
 'nature_sum': 41,
 'fragment_sum': 20,
 'geo_sum': 2,
 'object_sum': 49,
 'science_sum': 2,
 'graphic_sum': 16}

In [92]:
distribution_class_val = count_class(val_df)
distribution_class_val

{'animal_sum': 17,
 'human_sum': 9,
 'abs_sum': 2,
 'food_sum': 1,
 'art_arch_sum': 1,
 'nature_sum': 4,
 'fragment_sum': 2,
 'geo_sum': 'no positive class',
 'object_sum': 5,
 'science_sum': 'no positive class',
 'graphic_sum': 2}

In [93]:
distribution_class_test = count_class(df_test)
distribution_class_test

{'animal_sum': 9,
 'human_sum': 5,
 'abs_sum': 1,
 'food_sum': 'no positive class',
 'art_arch_sum': 1,
 'nature_sum': 2,
 'fragment_sum': 1,
 'geo_sum': 'no positive class',
 'object_sum': 3,
 'science_sum': 'no positive class',
 'graphic_sum': 1}

## Training the models


In [32]:
i = 0
grid_metric_results = []
for hp_dictionary in param_grid_macro_cont:
    print(f"PARAM {i}/{n_grids}")
    print(hp_dictionary)

    # try:

    MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE = set_hyperparams(hp_dictionary)

    train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
    valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
    test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN)

    train_data_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True,
        num_workers=0
    )

    val_data_loader = torch.utils.data.DataLoader(valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    test_data_loader = torch.utils.data.DataLoader(test_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    model = BERTClass()
    model.to(device)
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


    ckpt_path = "curr_ckpt"
    best_model_path = "best_model.pt"

    trained_model, best_metrics = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path, df_labels)

    metrics, f1, roc_auc, hamming, accuracy, test_loss, metrics_proba, metrics_targets = compute_test_metrics(test_data_loader)

    best_metrics["test"] = {}
    best_metrics["test"]["test_loss"] = test_loss
    best_metrics["test"]["test_f1"] = f1
    best_metrics["test"]["test_roc_auc"] = roc_auc
    best_metrics["test"]["test_hamming"] = hamming
    best_metrics["test"]["test_accuracy"] = accuracy
    best_metrics["test"]["test_proba"] = metrics_proba
    best_metrics["test"]["test_actual_classes"] = metrics_targets

    result = {
          'hyperparameters': hp_dictionary,
          'metrics': best_metrics
      }

    grid_metric_results.append(result)
    i = i + 1

PARAM 0/4
{'max-len': 128, 'learning_rate': 3e-05, 'batch_size': 8, 'num_train_epochs': 40}


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.10858585858585859, 'f1': 0.0, 'accuracy_score': 0.0}
              precision    recall  f1-score   support

  animal_sum       0.00      0.00      0.00        17
   human_sum       0.00      0.00      0.00         9
     abs_sum       0.00      0.00      0.00         2
    food_sum       0.00      0.00      0.00         1
art_arch_sum       0.00      0.00      0.00         1
  nature_sum       0.00      0.00      0.00         4
fragment_sum       0.00      0.00      0.00         2
     geo_sum       0.00      0.00      0.00         0
  object_sum       0.00      0.00      0.00         5
 science_sum       0.00      0.00      0.00         0
 graphic_sum       0.00      0.00      0.00         2

   micro avg       0.00      0.00      0.00      

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_s

############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2  Metrics   #############


{'roc_auc': 0.5936161802490282, 'hamming_loss': 0.12373737373737374, 'f1': 0.2898550724637681, 'accuracy_score': 0.2222222222222222}
              precision    recall  f1-score   support

  animal_sum       0.38      0.59      0.47        17
   human_sum       0.00      0.00      0.00         9
     abs_sum       0.00      0.00      0.00         2
    food_sum       0.00      0.00      0.00         1
art_arch_sum       0.00      0.00      0.00         1
  nature_sum       0.00      0.00      0.00         4
fragment_sum       0.00      0.00      0.00         2
     geo_sum       0.00      0.00      0.00         0
  object_sum       0.00      0.00      0.00         5
 science_sum       0.00      0.00      0.00         0
 graphic_sum       0.00      0.00      0.00         2

   micro avg       0.38      0.23      0.29        43
   m

...

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 40: Training End     #############
############# Epoch 40: Validation Start   #############
############# Epoch 40  Metrics   #############


{'roc_auc': 0.8868502536398973, 'hamming_loss': 0.03787878787878788, 'f1': 0.8192771084337349, 'accuracy_score': 0.7222222222222222}
              precision    recall  f1-score   support

  animal_sum       0.85      1.00      0.92        17
   human_sum       1.00      0.78      0.88         9
     abs_sum       1.00      0.50      0.67         2
    food_sum       1.00      1.00      1.00         1
art_arch_sum       1.00      1.00      1.00         1
  nature_sum       0.75      0.75      0.75         4
fragment_sum       0.00      0.00      0.00         2
     geo_sum       0.00      0.00      0.00         0
  object_sum       1.00      0.40      0.57         5
 science_sum       0.00      0.00      0.00         0
 graphic_sum       1.00      1.00      1.00         2

   micro avg       0.85      0.79      0.82        43
 

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'roc_auc': 0.9469063545150502, 'hamming_loss': 0.025974025974025976, 'f1': 0.8749999999999999, 'accuracy_score': 0.7619047619047619}
              precision    recall  f1-score   support

  animal_sum       0.75      1.00      0.86         9
   human_sum       1.00      0.80      0.89         5
     abs_sum       1.00      1.00      1.00         1
    food_sum       0.00      0.00      0.00         0
art_arch_sum       1.00      1.00      1.00         1
  nature_sum       0.67      1.00      0.80         2
fragment_sum       1.00      1.00      1.00         1
     geo_sum       0.00      0.00      0.00         0
  object_sum       1.00      0.67      0.80         3
 science_sum       0.00      0.00      0.00         0
 graphic_sum       1.00      1.00      1.00         1

   micro avg       0.84      0.91      0.87        23
   macro avg       0.67      0.68      0.67        23
weighted avg       0.87      0.91      0.88        23
 samples avg       0.90      0.97      0.91        23


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
with open("metrics_macro_conts_final.json", "w") as f:
    json.dump(grid_metric_results, f)

# Macro Determinants

In [94]:
param_grid_macro_dets = [
                        {
                        'max-len' : 128,
                        'learning_rate': 2e-5,
                        'batch_size': 8,
                        'num_train_epochs': 40
                                                },
                        {
                        'max-len' : 128,
                        'learning_rate': 5e-5,
                        'batch_size': 8,
                        'num_train_epochs': 40
                                                },
                        {
                        'max-len' : 128,
                        'learning_rate': 5e-5,
                        'batch_size': 12,
                        'num_train_epochs': 40
                                                }]
n_grids = len(param_grid_macro_dets)

In [95]:
df_name = dfs[3]
data_model = df_name.split("_")
data_model = "_".join(data_model[1:3]) #getting the name to name the different dictionaries
df = pd.read_csv(df_name)

test_proportion = 0.05
train_size = 0.9

train_df, val_df, df_test, df_labels  = pre_process_data(df, test_proportion, train_size)

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, 266 to 102
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   color_sum         380 non-null    float32
 1   threat_sum        380 non-null    float32
 2   fading_sum        380 non-null    float32
 3   form_sum          380 non-null    float32
 4   kinesthetics_sum  380 non-null    float32
dtypes: float32(5)
memory usage: 10.4 KB
None
['color_sum', 'threat_sum', 'fading_sum', 'form_sum', 'kinesthetics_sum']
Total amount of data: 380
Number of rows used to TRAIN: 326
Number of rows used to VALIDATE: 36
Number of rows used to TEST: 18


## Distribution of the classes in train, validate, test dfs

In [96]:
distribution_class_training = count_class(train_df)
distribution_class_training

{'color_sum': 118,
 'threat_sum': 22,
 'fading_sum': 36,
 'form_sum': 129,
 'kinesthetics_sum': 98}

In [97]:
distribution_class_val = count_class(val_df)
distribution_class_val

{'color_sum': 13,
 'threat_sum': 2,
 'fading_sum': 4,
 'form_sum': 14,
 'kinesthetics_sum': 11}

In [98]:
distribution_class_test = count_class(df_test)
distribution_class_test

{'color_sum': 7,
 'threat_sum': 1,
 'fading_sum': 2,
 'form_sum': 7,
 'kinesthetics_sum': 6}

In [36]:
i = 0
grid_metric_results = []
for hp_dictionary in param_grid_macro_dets:
    print(f"PARAM {i}/{n_grids}")
    print(hp_dictionary)

    # try:

    MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE = set_hyperparams(hp_dictionary)

    train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
    valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
    test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN)

    train_data_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True,
        num_workers=0
    )

    val_data_loader = torch.utils.data.DataLoader(valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    test_data_loader = torch.utils.data.DataLoader(test_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    model = BERTClass()
    model.to(device)
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


    ckpt_path = "curr_ckpt"
    best_model_path = "best_model.pt"

    trained_model, best_metrics = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path, df_labels)

    metrics, f1, roc_auc, hamming, accuracy, test_loss, metrics_proba, metrics_targets = compute_test_metrics(test_data_loader)

    best_metrics["test"] = {}
    best_metrics["test"]["test_loss"] = test_loss
    best_metrics["test"]["test_f1"] = f1
    best_metrics["test"]["test_roc_auc"] = roc_auc
    best_metrics["test"]["test_hamming"] = hamming
    best_metrics["test"]["test_accuracy"] = accuracy
    best_metrics["test"]["test_proba"] = metrics_proba
    best_metrics["test"]["test_actual_classes"] = metrics_targets

    result = {
          'hyperparameters': hp_dictionary,
          'metrics': best_metrics
      }

    grid_metric_results.append(result)
    i = i + 1

PARAM 0/3
{'max-len': 128, 'learning_rate': 2e-05, 'batch_size': 8, 'num_train_epochs': 40}


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1  Metrics   #############


{'roc_auc': 0.5, 'hamming_loss': 0.24444444444444444, 'f1': 0.0, 'accuracy_score': 0.0}
                  precision    recall  f1-score   support

       color_sum       0.00      0.00      0.00        13
      threat_sum       0.00      0.00      0.00         2
      fading_sum       0.00      0.00      0.00         4
        form_sum       0.00      0.00      0.00        14
kinesthetics_sum       0.00      0.00      0.00        11

       micro avg       0.00      0.00      0.00        44
       macro avg       0.00      0.00      0.00        44
    weighted avg       0.00      0.00      0.00        44
     samples avg       0.00      0.00      0.00        44

############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.013925 	Average Validati

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2  Metrics   #############


{'roc_auc': 0.633355614973262, 'hamming_loss': 0.20555555555555555, 'f1': 0.43076923076923074, 'accuracy_score': 0.2222222222222222}
                  precision    recall  f1-score   support

       color_sum       1.00      0.23      0.38        13
      threat_sum       0.00      0.00      0.00         2
      fading_sum       0.00      0.00      0.00         4
        form_sum       0.64      0.50      0.56        14
kinesthetics_sum       0.57      0.36      0.44        11

       micro avg       0.67      0.32      0.43        44
       macro avg       0.44      0.22      0.28        44
    weighted avg       0.64      0.32      0.40        44
     samples avg       0.32      0.30      0.30        44

############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.011682 	Average Validation Loss: 

...

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 40: Training End     #############
############# Epoch 40: Validation Start   #############


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 40  Metrics   #############


{'roc_auc': 0.8231951871657753, 'hamming_loss': 0.12777777777777777, 'f1': 0.735632183908046, 'accuracy_score': 0.5833333333333334}
                  precision    recall  f1-score   support

       color_sum       0.61      0.85      0.71        13
      threat_sum       1.00      0.50      0.67         2
      fading_sum       0.00      0.00      0.00         4
        form_sum       0.79      0.79      0.79        14
kinesthetics_sum       0.90      0.82      0.86        11

       micro avg       0.74      0.73      0.74        44
       macro avg       0.66      0.59      0.60        44
    weighted avg       0.70      0.73      0.70        44
     samples avg       0.74      0.73      0.72        44

############# Epoch 40: Validation End     #############
Epoch: 40 	Avgerage Training Loss: 0.000130 	Average Validation Loss: 0.190060
############# Epoch 40  Done   #############

{'roc_auc': 0.6427644386761843, 'hamming_loss': 0.255

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
with open("metrics_macro_dets_final.json", "w") as f:
    json.dump(grid_metric_results, f)